In [8]:
import pandas as pd
import dask

In [4]:
!pip install dask[dataframe]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 140 kB 6.6 MB/s 


In [9]:
import dask.dataframe as dd

# Approach #1 to read csv fie - Pandas

In [6]:
%time pd_dataframe = pd.read_csv('people_data.csv')

CPU times: user 13.8 s, sys: 1.58 s, total: 15.4 s
Wall time: 15.4 s


# Approach #2 to read csv file - dask

In [11]:
%time dd_dataframe = dd.read_csv('people_data.csv', quoting=3, error_bad_lines=False)

CPU times: user 34.5 ms, sys: 1.15 ms, total: 35.7 ms
Wall time: 174 ms


/usr/local/lib/python3.7/dist-packages/dask/dataframe/io/csv.py:578: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  **kwargs


## As seen above, the computational efficiency is much higher by using Dask

# Approach #3 to read csv file - modin

In [13]:
!pip install ray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 54.5 MB 125 kB/s 
     |████████████████████████████████| 10.1 MB 35.6 MB/s 
     |████████████████████████████████| 4.1 MB 45.6 MB/s 
     |████████████████████████████████| 144 kB 55.4 MB/s 
     |████████████████████████████████| 461 kB 50.6 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.47.0
    Uninstalling grpcio-1.47.0:
      Successfully uninstalled grpcio-1.47.0


In [14]:
!pip install modin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 761 kB 7.3 MB/s 


In [15]:
import modin.pandas as mpd
import ray

In [16]:
%time mpd_dataframe = mpd.read_csv('people_data.csv')

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()



CPU times: user 2.7 s, sys: 1.11 s, total: 3.81 s
Wall time: 32.9 s


In [17]:
%%writefile utilities.py
import logging
import os
import subprocess
import yaml
import pandas as pd
import datetime 
import gc
import re


################
# File Reading #
################

def read_config_file(filepath):
    with open(filepath, 'r') as stream:
        try:
            return yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            logging.error(exc)


def replacer(string, char):
    pattern = char + '{2,}'
    string = re.sub(pattern, char, string) 
    return string

def col_header_val(df,table_config):
    '''
    replace whitespaces in the column
    and standardized column names
    '''
    df.columns = df.columns.str.lower()
    df.columns = df.columns.str.replace('[^\w]','_',regex=True)
    df.columns = list(map(lambda x: x.strip('_'), list(df.columns)))
    df.columns = list(map(lambda x: replacer(x,'_'), list(df.columns)))
    expected_col = list(map(lambda x: x.lower(),  table_config['columns']))
    expected_col.sort()
    df.columns =list(map(lambda x: x.lower(), list(df.columns)))
    df = df.reindex(sorted(df.columns), axis=1)
    if len(df.columns) == len(expected_col) and list(expected_col)  == list(df.columns):
        print("column name and column length validation passed")
        return 1
    else:
        print("column name and column length validation failed")
        mismatched_columns_file = list(set(df.columns).difference(expected_col))
        print("Following File columns are not in the YAML file",mismatched_columns_file)
        missing_YAML_file = list(set(expected_col).difference(df.columns))
        print("Following YAML columns are not in the file uploaded",missing_YAML_file)
        logging.info(f'df columns: {df.columns}')
        logging.info(f'expected columns: {expected_col}')
        return 0

def data_info(df,table_congif,test_data):

  rows = len(df.axes[0])
  cols = len(df.axes[1])
  file_size = os.path.getsize(test_data)

  return rows, cols, file_size

Writing utilities.py


In [18]:
%%writefile file.yaml
file_type: csv
dataset_name: testfile
file_name: people_data
table_name: edsurv
inbound_delimiter: ","
outbound_delimiter: "|"
skip_leading_rows: 1
columns: 
    - Name
    - Job
    - Address
    - Phone
    - Email

Writing file.yaml


In [19]:
# Read config file
import utilities as util
config_data = util.read_config_file("file.yaml")

In [20]:
config_data['columns']

['Name', 'Job', 'Address', 'Phone', 'Email']

In [21]:
config_data

{'columns': ['Name', 'Job', 'Address', 'Phone', 'Email'],
 'dataset_name': 'testfile',
 'file_name': 'people_data',
 'file_type': 'csv',
 'inbound_delimiter': ',',
 'outbound_delimiter': '|',
 'skip_leading_rows': 1,
 'table_name': 'edsurv'}

In [22]:
#reading file using yaml file
file_type = config_data['file_type']
source_file = "./" + config_data['file_name'] + f'.{file_type}'

df = pd.read_csv(source_file,config_data['inbound_delimiter'])
#df.head()

In [23]:
#write file in '|' separated gz format

import gzip
import shutil

dfgzip = pd.read_csv(r'people_data.csv') #read inputfile in a dataframe
dfgzip.to_csv(r'people_data_2.csv', sep = config_data['outbound_delimiter'])

with open('people_data_2.csv', 'rb') as f_in:
    with gzip.open('people_data_2.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [24]:
#Validate number of columns in config file and the dataframe

rows, cols, file_size = util.data_info(df,config_data, "./" + config_data['file_name'] + f'.{file_type}')

if cols == len(config_data['columns']):
    print('Validation Passed! The no. of columns in the config file is equal to the no. of columns in the dataframe')
else:
    print('Validation Failed! The no. of columns in the config file is NOT equal to the no. of columns in the dataframe')
#print(cols)
#print(len(config_data['columns']))

Validation Failed! The no. of columns in the config file is NOT equal to the no. of columns in the dataframe


In [25]:
#Validate column headers of the config file with the dataframe

util.col_header_val(df,config_data)

column name and column length validation failed
Following File columns are not in the YAML file ['unnamed_0']
Following YAML columns are not in the file uploaded []


0

## As observed, the column length validation failed as the extra 1st column 'unnamed_0' is not mentioned in the yaml file

In [26]:
# Comparing columns of the dataframe and the yaml file

print("columns of loaded DataFrame are:" ,df.columns)
print("columns of YAML file are:" ,config_data['columns'])

columns of loaded DataFrame are: Index(['unnamed_0', 'name', 'job', 'address', 'phone', 'email'], dtype='object')
columns of YAML file are: ['Name', 'Job', 'Address', 'Phone', 'Email']


In [27]:
#File Summary

print('No. of rows in the file:', rows)
print('No. of columns in the file:', cols)
print('File Size:', file_size, 'bytes')


No. of rows in the file: 5000000
No. of columns in the file: 6
File Size: 655399736 bytes


In [ ]:
# Code to generate the dataset for this assignment
# the code was run in google colab

from faker import Faker
import pandas as pd
fake = Faker()

def generate_person():
  return {
      'Name': fake.name(),
      'Job': fake.job(),
      'Address': fake.address(),
      'Phone': fake.phone_number(),
      'Email': fake.email()
  }

values = [generate_person() for i in range(5000000)]
people = pd.DataFrame(values)

people.to_csv(r'/content/people_data.csv')